In [116]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer 

In [117]:
nltk.download('wordnet')
nltk.download('cmudict')

[nltk_data] Downloading package wordnet to /Users/farhan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package cmudict to /Users/farhan/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


True

In [118]:
subtlex_path = '/Users/farhan/Desktop/Baycrest Documents/Aphasia_Study/Aphasia_STM_stim_generation/Syllable_Project/SUBTLEX-US_frequency_list.csv'
goodwords_path = '/Users/farhan/Desktop/Baycrest Documents/Aphasia_Study/Aphasia_STM_stim_generation/Syllable_Project/goodwords_cmusyllables.csv'

In [119]:
subtlex = pd.read_csv(subtlex_path)
goodwords = pd.read_csv(goodwords_path)

In [120]:
cdict = cmudict.dict()

def syllable_count(word):
    try:
        return [len(list(y for y in x if y[-1].isdigit())) for x in cdict[word.lower()]][0]
    except KeyError:
        return -1

Narrow down to between 4 and 5 frequency (4 < Zipfvalue < 5)

In [121]:
freq_corrected_subtlex = subtlex.loc[subtlex.Zipfvalue < 5]
freq_corrected_subtlex = freq_corrected_subtlex.loc[subtlex.Zipfvalue > 4].reset_index(drop = True)

In [124]:
freq_corrected_subtlex["syllables"] = freq_corrected_subtlex.apply(lambda x: syllable_count(x['Word']), axis=1)
pd.set_option('display.max_rows', None)

lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize("bats")
freq_corrected_subtlex['lemma'] = freq_corrected_subtlex.apply(lambda x: lemmatizer.lemmatize(x['Word']), axis=1)

In [126]:
freq_corrected_subtlex = freq_corrected_subtlex.loc[freq_corrected_subtlex.syllables <= 2].reset_index(drop = True)
freq_corrected_subtlex

,Word,FREQcount,CDcount,FREQlow,Cdlow,SUBTLWF,Lg10WF,SUBTLCD,Lg10CD,Dom_PoS_SUBTLEX,Freq_dom_PoS_SUBTLEX,Percentage_dom_PoS,All_PoS_SUBTLEX,All_freqs_SUBTLEX,Zipfvalue,syllables,lemma
0,aah,2688,634,52,37,52.71,3.4296,7.56,2.8028,Interjection,2657.0,1.00,Interjection,2657,4.721425,-1,aah
1,aaron,747,162,0,0,14.65,2.8739,1.93,2.2122,Name,744.0,1.00,Name,744,4.165736,2,aaron
2,abby,637,102,0,0,12.49,2.8048,1.22,2.0128,Name,632.0,1.00,Name,632,4.096655,2,abby
3,aboard,1358,709,1335,706,26.63,3.1332,8.45,2.8513,Adverb,1006.0,0.75,Adverb.Preposition,1006.342,4.425054,2,aboard
4,above,2493,1778,2350,1703,48.88,3.3969,21.20,3.2502,Preposition,1770.0,0.71,Preposition.Adverb.Adjective,1770.608.110,4.688731,2,above
5,abuse,523,409,503,396,10.25,2.7193,4.88,2.6128,Noun,404.0,0.77,Noun.Verb.Name,404.118.1,4.011165,2,abuse
6,accent,621,447,595,438,12.18,2.7938,5.33,2.6513,Noun,609.0,0.98,Noun.Verb,609.12,4.085625,2,accent
7,accept,2692,1861,2598,1820,52.78,3.4302,22.19,3.2700,Verb,2692.0,1.00,Verb.Noun,2692.1,4.722070,2,accept
8,access,1615,923,1483,879,31.67,3.2084,11.00,2.9657,Noun,1318.0,0.82,Noun.Verb,1318.283,4.500275,2,access
9,account,2280,1386,2253,1375,44.71,3.3581,16.52,3.1421,Noun,1587.0,0.70,Noun.Preposition.Verb,1587.355.335,4.649959,2,account
